## 环境配置

第一步：下载mindspore2.2.13并放置于当前目录下

下载地址为：https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.2.13/MindSpore/unified/aarch64/mindspore-2.2.13-cp39-cp39-linux_aarch64.whl
使用obs上传到当前目录

第二步：安装MindSpore框架和MindNLP套件

In [3]:
!pip install mindspore-2.2.13-cp39-cp39-linux_aarch64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Processing ./mindspore-2.2.13-cp39-cp39-linux_aarch64.whl
  Attempting uninstall: mindspore
    Found existing installation: mindspore 2.2.0
    Uninstalling mindspore-2.2.0:
      Successfully uninstalled mindspore-2.2.0
You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python3.9 -m pip install --upgrade pip' command.


In [4]:
!pip install mindnlp

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
     |████████████████████████████████| 5.7 MB 36.3 MB/s eta 0:00:01
     |████████████████████████████████| 316 kB 49.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 63.7 MB/s eta 0:00:01
     |████████████████████████████████| 542 kB 64.4 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 51.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 33.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 52.4 MB/s eta 0:00:01
     |████████████████████████████████| 38.1 MB 58.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 36.0 MB/s eta 0:00:01
     |████████████████████████████████| 401 kB 63.1 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 50.5 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 66.6 MB/s eta 0:00:01
     |████████████████████████████████| 220 kB 59.3 MB/s eta 0:00:01
     |█████████████████

#### 注：MindNLP whl包下载链接为：[MindNLP](https://repo.mindspore.cn/mindspore-lab/mindnlp/newest/any/)

# 基于 MindSpore 实现 BERT 对话情绪识别


## 模型简介

BERT全称是来自变换器的双向编码器表征量（Bidirectional Encoder Representations from Transformers），它是Google于2018年末开发并发布的一种新型语言模型。与BERT模型相似的预训练语言模型例如问答、命名实体识别、自然语言推理、文本分类等在许多自然语言处理任务中发挥着重要作用。模型是基于Transformer中的Encoder并加上双向的结构，因此一定要熟练掌握Transformer的Encoder的结构。

BERT模型的主要创新点都在pre-train方法上，即用了Masked Language Model和Next Sentence Prediction两种方法分别捕捉词语和句子级别的representation。

在用Masked Language Model方法训练BERT的时候，随机把语料库中15%的单词做Mask操作。对于这15%的单词做Mask操作分为三种情况：80%的单词直接用[Mask]替换、10%的单词直接替换成另一个新的单词、10%的单词保持不变。

因为涉及到Question Answering (QA) 和 Natural Language Inference (NLI)之类的任务，增加了Next Sentence Prediction预训练任务，目的是让模型理解两个句子之间的联系。与Masked Language Model任务相比，Next Sentence Prediction更简单些，训练的输入是句子A和B，B有一半的几率是A的下一句，输入这两个句子，BERT模型预测B是不是A的下一句。

BERT预训练之后，会保存它的Embedding table和12层Transformer权重（BERT-BASE）或24层Transformer权重（BERT-LARGE）。使用预训练好的BERT模型可以对下游任务进行Fine-tuning，比如：文本分类、相似度判断、阅读理解等。

对话情绪识别（Emotion Detection，简称EmoTect），专注于识别智能对话场景中用户的情绪，针对智能对话场景中的用户文本，自动判断该文本的情绪类别并给出相应的置信度，情绪类型分为积极、消极、中性。 对话情绪识别适用于聊天、客服等多个场景，能够帮助企业更好地把握对话质量、改善产品的用户交互体验，也能分析客服服务质量、降低人工质检成本。

下面以一个文本情感分类任务为例子来说明BERT模型的整个应用过程。

In [5]:
import os

import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn, context

from mindnlp._legacy.engine import Trainer, Evaluator
from mindnlp._legacy.engine.callbacks import CheckpointCallback, BestModelCallback
from mindnlp._legacy.metrics import Accuracy

[WARNING] ME(2190:281473358934192,MainProcess):2024-06-05-15:37:35.976.539 [mindspore/run_check/_check_version.py:357] MindSpore version 2.2.13 and Ascend AI software package (Ascend Data Center Solution)version 7.0 does not match, the version of software package expect one of ['7.1']. Please refer to the match info on: https://www.mindspore.cn/install
/usr/local/Ascend/ascend-toolkit/7.0.RC1/python/site-packages/tbe/tvm/contrib/ccec.py:766: DeprecationWarning: invalid escape sequence \L
  if not dirpath.find("AppData\Local\Temp"):
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/classifier/transdata/transdata_classifier.py:222: DeprecationWarning: invalid escape sequence \B
  """
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/vector/transdata/common/graph/transdata_graph_info.py:140: DeprecationWarning: invalid escape sequence \c
  """
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499

In [6]:
# prepare dataset
class SentimentDataset:
    """Sentiment Dataset"""

    def __init__(self, path):
        self.path = path
        self._labels, self._text_a = [], []
        self._load()

    def _load(self):
        with open(self.path, "r", encoding="utf-8") as f:
            dataset = f.read()
        lines = dataset.split("\n")
        for line in lines[1:-1]:
            label, text_a = line.split("\t")
            self._labels.append(int(label))
            self._text_a.append(text_a)

    def __getitem__(self, index):
        return self._labels[index], self._text_a[index]

    def __len__(self):
        return len(self._labels)

## 数据集

这里提供一份已标注的、经过分词预处理的机器人聊天数据集，来自于百度飞桨团队。数据由两列组成，以制表符（'\t'）分隔，第一列是情绪分类的类别（0表示消极；1表示中性；2表示积极），第二列是以空格分词的中文文本，如下示例，文件为 utf8 编码。

label--text_a

0--谁骂人了？我从来不骂人，我骂的都不是人，你是人吗 ？

1--我有事等会儿就回来和你聊

2--我见到你很高兴谢谢你帮我

这部分主要包括数据集读取，数据格式转换，数据 Tokenize 处理和 pad 操作。

In [7]:
# download dataset
!wget https://baidu-nlp.bj.bcebos.com/emotion_detection-dataset-1.0.0.tar.gz -O emotion_detection.tar.gz
!tar xvf emotion_detection.tar.gz

--2024-06-05 15:38:48--  https://baidu-nlp.bj.bcebos.com/emotion_detection-dataset-1.0.0.tar.gz
Resolving proxy-notebook.modelarts.com (proxy-notebook.modelarts.com)... 192.168.0.33
Connecting to proxy-notebook.modelarts.com (proxy-notebook.modelarts.com)|192.168.0.33|:8083... connected.
Proxy request sent, awaiting response... 200 OK
Length: 1710581 (1.6M) [application/x-gzip]
Saving to: ‘emotion_detection.tar.gz’

emotion_detection.t 100%[===================>]   1.63M  6.93MB/s    in 0.2s    

2024-06-05 15:38:49 (6.93 MB/s) - ‘emotion_detection.tar.gz’ saved [1710581/1710581]

data/
data/test.tsv
data/infer.tsv
data/dev.tsv
data/train.tsv
data/vocab.txt


### 数据加载和数据预处理

新建 process_dataset 函数用于数据加载和数据预处理，具体内容可见下面代码注释。

In [8]:
import numpy as np

def process_dataset(source, tokenizer, max_seq_len=64, batch_size=32, shuffle=True):
    is_ascend = mindspore.get_context('device_target') == 'Ascend'

    column_names = ["label", "text_a"]
    
    dataset = GeneratorDataset(source, column_names=column_names, shuffle=shuffle)
    # transforms
    type_cast_op = transforms.TypeCast(mindspore.int32)
    def tokenize_and_pad(text):
        if is_ascend:
            tokenized = tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_len)
        else:
            tokenized = tokenizer(text)
        return tokenized['input_ids'], tokenized['attention_mask']
    # map dataset
    dataset = dataset.map(operations=tokenize_and_pad, input_columns="text_a", output_columns=['input_ids', 'attention_mask'])
    dataset = dataset.map(operations=[type_cast_op], input_columns="label", output_columns='labels')
    # batch dataset
    if is_ascend:
        dataset = dataset.batch(batch_size)
    else:
        dataset = dataset.padded_batch(batch_size, pad_info={'input_ids': (None, tokenizer.pad_token_id),
                                                         'attention_mask': (None, 0)})

    return dataset

昇腾NPU环境下暂不支持动态Shape，数据预处理部分采用静态Shape处理：

In [9]:
from mindnlp.transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

100%|██████████| 49.0/49.0 [00:00<00:00, 65.2kB/s]
107kB [00:00, 651kB/s] 
263kB [00:00, 1.32MB/s]
624B [00:00, 654kB/s]                    


In [13]:
tokenizer.pad_token_id

0

In [14]:
dataset_train = process_dataset(SentimentDataset("data/train.tsv"), tokenizer)
dataset_val = process_dataset(SentimentDataset("data/dev.tsv"), tokenizer)
dataset_test = process_dataset(SentimentDataset("data/test.tsv"), tokenizer, shuffle=False)

In [15]:
dataset_train.get_col_names()

['input_ids', 'attention_mask', 'labels']

In [16]:
print(next(dataset_train.create_tuple_iterator()))

[Tensor(shape=[32, 64], dtype=Int64, value=
[[ 101, 6821, 3416 ...    0,    0,    0],
 [ 101, 1962, 2769 ...    0,    0,    0],
 [ 101,  809, 1400 ...    0,    0,    0],
 ...
 [ 101, 2769, 1355 ...    0,    0,    0],
 [ 101,  872, 2218 ...    0,    0,    0],
 [ 101, 5440, 6407 ...    0,    0,    0]]), Tensor(shape=[32, 64], dtype=Int64, value=
[[1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 ...
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0],
 [1, 1, 1 ... 0, 0, 0]]), Tensor(shape=[32], dtype=Int32, value= [0, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 
 1, 1, 1, 2, 2, 2, 1, 1])]


## 模型构建

通过 BertForSequenceClassification 构建用于情感分类的 BERT 模型，加载预训练权重，设置情感三分类的超参数自动构建模型。后面对模型采用自动混合精度操作，提高训练的速度，然后实例化优化器，紧接着实例化评价指标，设置模型训练的权重保存策略，最后就是构建训练器，模型开始训练。

In [17]:
from mindnlp.transformers import BertForSequenceClassification, BertModel
from mindnlp._legacy.amp import auto_mixed_precision

# set bert config and define parameters for training
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=3)
model = auto_mixed_precision(model, 'O1')

optimizer = nn.Adam(model.trainable_params(), learning_rate=2e-5)

100%|██████████| 392M/392M [00:45<00:00, 9.11MB/s] 
The following parameters in checkpoint files are not loaded:
['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
The following parameters in models are missing parameter:
['classifier.weight', 'classifier.bias']


In [19]:
metric = Accuracy()
# define callbacks to save checkpoints
ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='bert_emotect', epochs=1, keep_checkpoint_max=2)
best_model_cb = BestModelCallback(save_path='checkpoint', ckpt_name='bert_emotect_best', auto_load=True)

trainer = Trainer(network=model, train_dataset=dataset_train,
                  eval_dataset=dataset_val, metrics=metric,
                  epochs=5, optimizer=optimizer, callbacks=[ckpoint_cb, best_model_cb])

In [20]:
# start training
trainer.run(tgt_columns="labels")

The train will start from the checkpoint saved in 'checkpoint'.


Epoch 0:   0%|          | 0/302 [00:00<?, ?it/s]

Epoch 0: 100%|█████████▉| 301/302 [03:47<00:00,  1.99it/s, loss=0.33261508]

Epoch 0: 100%|██████████| 302/302 [03:52<00:00,  1.30it/s, loss=0.332153]  


Checkpoint: 'bert_emotect_epoch_0.ckpt' has been saved in epoch: 0.


...

Evaluate Score: {'Accuracy': 0.9898148148148148}
---------------Best Model: 'bert_emotect_best.ckpt' has been saved in epoch: 3.---------------


Epoch 4: 100%|██████████| 302/302 [02:41<00:00,  1.88it/s, loss=0.060223028]


The maximum number of stored checkpoints has been reached.
Checkpoint: 'bert_emotect_epoch_4.ckpt' has been saved in epoch: 4.


Evaluate: 100%|██████████| 34/34 [00:04<00:00,  7.06it/s]


Evaluate Score: {'Accuracy': 0.9842592592592593}
Loading best model from 'checkpoint' with '['Accuracy']': [0.9898148148148148]...
---------------The model is already load the best model from 'bert_emotect_best.ckpt'.---------------


## 模型验证

将验证数据集加再进训练好的模型，对数据集进行验证，查看模型在验证数据上面的效果，此处的评价指标为准确率。

In [21]:
evaluator = Evaluator(network=model, eval_dataset=dataset_test, metrics=metric)
evaluator.run(tgt_columns="labels")

Evaluate:  97%|█████████▋| 32/33 [00:04<00:00,  7.93it/s]

Evaluate: 100%|██████████| 33/33 [00:15<00:00,  2.11it/s]

Evaluate Score: {'Accuracy': 0.9025096525096525}


## 模型推理

遍历推理数据集，将结果与标签进行统一展示。

In [22]:
dataset_infer = SentimentDataset("data/infer.tsv")

In [17]:
def predict(text, label=None):
    label_map = {0: "消极", 1: "中性", 2: "积极"}

    text_tokenized = Tensor([tokenizer(text).input_ids])
    logits = model(text_tokenized)
    predict_label = logits[0].asnumpy().argmax()
    info = f"inputs: '{text}', predict: '{label_map[predict_label]}'"
    if label is not None:
        info += f" , label: '{label_map[label]}'"
    print(info)

In [18]:
from mindspore import Tensor

for label, text in dataset_infer:
    predict(text, label)

inputs: '我 要 客观', predict: '中性' , label: '中性'

inputs: '靠 你 真是 说 废话 吗', predict: '消极' , label: '消极'

inputs: '口嗅 会', predict: '中性' , label: '中性'

inputs: '每次 是 表妹 带 窝 飞 因为 窝路痴', predict: '中性' , label: '中性'

inputs: '别说 废话 我 问 你 个 问题', predict: '消极' , label: '消极'

inputs: '4967 是 新加坡 那 家 银行', predict: '中性' , label: '中性'

inputs: '是 我 喜欢 兔子', predict: '积极' , label: '积极'

inputs: '你 写 过 黄山 奇石 吗', predict: '中性' , label: '中性'

inputs: '一个一个 慢慢来', predict: '中性' , label: '中性'

inputs: '我 玩 过 这个 一点 都 不 好玩', predict: '消极' , label: '消极'

inputs: '网上 开发 女孩 的 QQ', predict: '中性' , label: '中性'

inputs: '背 你 猜 对 了', predict: '中性' , label: '中性'

inputs: '我 讨厌 你 ， 哼哼 哼 。 。', predict: '消极' , label: '消极'


## 自定义推理数据集

自己输入推理数据，展示模型的泛化能力。

In [19]:
predict("家人们咱就是说一整个无语住了 绝绝子叠buff")

inputs: '家人们咱就是说一整个无语住了 绝绝子叠buff', predict: '中性'
